In [29]:
import pandas as pd
import yfinance as yf
import pickle
import datetime

from os.path import exists

In [30]:
tickers = ['jpm', 'tsla', 'SPY', 'BRK-B', 'INTC', 'SSO', 'ROM']

In [31]:
# today's date for the end date
end_dt = datetime.date.today()

# loop through each ticker
for sym in tickers:
    
    # build file name and read in cache if it exists
    file_name = 'data/' + sym + '.pkl'
    df_prev = pd.DataFrame()
    if exists( file_name ):
        df_prev = pd.read_pickle( file_name )
        start_dt = df_prev.index.max()
    else:
        start_dt = '2000-01-01'
    print( sym + ' ' + str(start_dt) + ' to ' + str(end_dt) )
    
    # download the data we need
    df = yf.download(sym,
              start=start_dt, 
              end=end_dt, 
              progress=False)
    if len( df_prev ) > 0:        
        # merge the dataframes
        df_index = df.index
        df_prev_index = df_prev.index
        mask = ~df_prev_index.isin(df_index)
        #result = pd_prev.loc[mask]
        df = pd.concat([df_prev.loc[mask],df])
    
    df['close_30day'] = df['Close'].rolling(window=30).mean()
    df['close_60day'] = df['Close'].rolling(window=60).mean()
    df['close_90day'] = df['Close'].rolling(window=90).mean()
    df.to_pickle( file_name )
    

jpm 2022-08-03 00:00:00 to 2022-08-04
tsla 2022-08-03 00:00:00 to 2022-08-04
SPY 2022-08-03 00:00:00 to 2022-08-04
BRK-B 2022-08-03 00:00:00 to 2022-08-04
INTC 2022-08-03 00:00:00 to 2022-08-04
SSO 2022-08-03 00:00:00 to 2022-08-04
ROM 2022-08-03 00:00:00 to 2022-08-04
